In [1]:
#hide
#default_exp test
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *
from nbdev.export import notebook2script

# Tests

> Functions and scripts for performing automated testing on a KiCad project

* toc: true

In [2]:
#export
import os
import subprocess

from fastcore.script import *
from nbdev.test import *
from nbdev.test import nbglob, num_cpus, parallel, _test_one, Path
import pandas as pd
import pandera as pa
from kicad_helpers import *
from kicad_helpers.utilities import _set_root, _print_cmd_output

In [3]:
#hide
root = os.path.join(get_git_root("."), "_temp")
setup_test_repo(root)

### ERC and DRC tests

ERC and DRC test are performed using [KiBot](https://github.com/INTI-CMNB/KiBot). These tests can be configured by editing the `erc.yaml` and `drc.yaml` files in the `.kicad_helpers_config` directory. For example, ERC warnings are treated as errors by default, but this behavior can be changed by replacing the following line in `.kicad_helpers_config/erc.yaml`:

```yaml
  erc_warnings: true
```

with

```yaml
  erc_warnings: false
```

You can also filter out specific errors/warnings using filters. Refer to the [KiBot documentation](https://github.com/INTI-CMNB/KiBot#filtering-drc-and-erc-errors) for details.

In [4]:
#export
def test_erc(root="."):
    returncode = 0
    config = f".kicad_helpers_config/erc.yaml"
    try:
        output = run_kibot_docker(config=config, root=root)
    except subprocess.CalledProcessError as e:
        returncode = e.returncode
        print(e.output.decode("utf-8"))
        print(f"returncode = { returncode }")
    erc_path = os.path.join(root, get_project_name(root) + ".erc")    
    if os.path.exists(erc_path):
        os.remove(erc_path)
    assert returncode == 0

In [5]:
test_erc(root)

In [6]:
#export
def test_drc(root="."):
    returncode = 0
    config = f".kicad_helpers_config/drc.yaml"
    try:
        output = run_kibot_docker(config=config, root=root)
    except subprocess.CalledProcessError as e:
        returncode = e.returncode
        print(e.output.decode("utf-8"))
        print(f"returncode = { returncode }")
    drc_path = os.path.join(root, "drc_result.rpt")
    if os.path.exists(drc_path):
        os.remove(drc_path)
    assert returncode == 0

In [7]:
#hide

# Checkout `.kicad_helpers_config/drc.yaml` because is contains filters
# overriding the default template installed via `kh_update --overwrite`
print(subprocess.check_output(f"cd { root } && git checkout .kicad_helpers_config/drc.yaml", shell=True).decode("utf-8"))

Updated 0 paths from the index


In [8]:
test_drc(root)

### BOM validation

In [9]:
#export
def validate_bom(root="."):
    df = pd.read_csv(get_bom_path(root))
    schema = pa.DataFrameSchema({
        "Refs": pa.Column(str),
        "Quantity": pa.Column(int),
        "MPN": pa.Column(str),
        "Manufacturer": pa.Column(str),
        "datasheet": pa.Column(str, nullable=True),
        "footprint": pa.Column(str),
        "value": pa.Column(str),
    })

    return schema.validate(df)

In [10]:
validate_bom(root)

,Refs,Quantity,MPN,Manufacturer,datasheet,footprint,value
0,"C1, C3, C5, C6, C8-C13, C16",11,CL21B104KBCNNNC,Samsung,NaN,Capacitors_SMD:C_0805,0.1uF
1,"C14, C15",2,CL21C151JBANNNC,Samsung,NaN,Capacitors_SMD:C_0805,150pF
2,"C2, C4",2,T491D336K020AT,KEMET,NaN,Sci-Bots:SM2917,33uF
3,C7,1,CL21B103KCANNNC,Samsung,NaN,Capacitors_SMD:C_0805,0.01uF
4,CH0-CH39,40,AQW214EAZ,Panasonic,NaN,SMD_Packages:DIP-8_SMD,AQW214
5,DS1,1,150080BS75000,Würth Elektronik,NaN,LEDs:LED_0805,+3.3V_PWR
6,FB1-FB5,5,742792040,Würth Elektronik,NaN,Resistors_SMD:R_0805,FERRITE
7,JP1,1,DNP,DNP,NaN,Resistors_SMD:R_0805,JUMPER
8,P1,1,DNP,DNP,NaN,Pin_Headers:Pin_Header_Angled_1x06,CONN_01X06
9,P2,1,DNP,DNP,NaN,Pin_Headers:Pin_Header_Straight_2x03,CONN_01X06


### Test notebooks

In [11]:
#export
@call_parse
def test_notebooks(fname:Param("A notebook name or glob to convert", str)=None,
                   flags:Param("Space separated list of flags", str)=None,
                   n_workers:Param("Number of workers to use", int)=None,
                   verbose:Param("Print errors along the way", bool_arg)=True,
                   timing:Param("Timing each notebook to see the ones are slow", bool)=False,
                   pause:Param("Pause time (in secs) between notebooks to avoid race conditions", float)=0.5,
                   root:Param("project root directory", str)="."):
    """Test all notebooks matching `fname` in parallel, passing along `flags`"""
    root = _set_root(root)
    if flags is not None: flags = flags.split(' ')
    if fname is None:
        fname = os.path.join(root, "tests", "*.ipynb")
    files = nbglob(fname, recursive=False)
    files = [Path(f).absolute() for f in sorted(files)]
    assert len(files) > 0, "No files to test found."
    if n_workers is None: n_workers = 0 if len(files)==1 else min(num_cpus(), 8)
    # make sure we are inside the tests folder
    os.chdir(os.path.join(root, "tests"))
    results = parallel(_test_one, files, flags=flags, verbose=verbose, n_workers=n_workers, pause=pause)
    passed,times = [r[0] for r in results],[r[1] for r in results]
    if all(passed): print("All tests are passing!")
    else:
        msg = "The following notebooks failed:\n"
        raise Exception(msg + '\n'.join([f.name for p,f in zip(passed,files) if not p]))
    if timing:
        for i,t in sorted(enumerate(times), key=lambda o:o[1], reverse=True):
            print(f"Notebook {files[i].name} took {int(t)} seconds")

Run all tests in the `tests` directory.

```sh
> kh_test
```

In [12]:
#hide_input
_print_cmd_output(f"kh_test --root { root }")

testing /home/ryan/dev/python/kicad-helpers/_temp/tests/Tests.ipynb
All tests are passing!



In [13]:
#hide
notebook2script()

Converted 00_actions.ipynb.
Converted 01_test.ipynb.
Converted 02_utilities.ipynb.
Converted index.ipynb.
